In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [23]:
# File from http://download.geofabrik.de/north-america/us/new-york.html
df = gpd.read_file('../data/osm/gis_osm_roads_free_1.shp')

In [24]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1266805 entries, 0 to 1266804
Data columns (total 11 columns):
 #   Column    Non-Null Count    Dtype   
---  ------    --------------    -----   
 0   osm_id    1266805 non-null  object  
 1   code      1266805 non-null  int64   
 2   fclass    1266805 non-null  object  
 3   name      391991 non-null   object  
 4   ref       54642 non-null    object  
 5   oneway    1266805 non-null  object  
 6   maxspeed  1266805 non-null  int64   
 7   layer     1266805 non-null  int64   
 8   bridge    1266805 non-null  object  
 9   tunnel    1266805 non-null  object  
 10  geometry  1266805 non-null  geometry
dtypes: geometry(1), int64(3), object(7)
memory usage: 106.3+ MB


In [25]:
# Remove unnecessary columns
df = df[['name', 'code', 'geometry']]

In [26]:
df.head(3)

,name,code,geometry
0,None,5131,"LINESTRING (-78.76634 42.95415, -78.76623 42.9..."
1,Niagara Expressway,5111,"LINESTRING (-79.03437 43.15172, -79.03391 43.1..."
2,South Grand Island Bridge,5111,"LINESTRING (-78.94184 43.00122, -78.94136 43.0..."


In [29]:
# Only keep car based roads as film permits do not track pedestrian paths
df = df.loc[(df['code'] >= 5111) & (df['code'] <= 5125) & (df['code'] != 5124)]

# Drop nameless and nonsense name roads
df = df.loc[(df['name'].notnull()) & (df['name'] != '-')]

# Drop 'code' column
df.drop(columns='code', inplace=True)

In [30]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 351305 entries, 1 to 1266771
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   name      351305 non-null  object  
 1   geometry  351305 non-null  geometry
dtypes: geometry(1), object(1)
memory usage: 8.0+ MB


In [31]:
# NYC borough boundaries
boros = gpd.read_file('./data/nyc/boros.geojson')

# NYC MultiPolygon object
nyc = boros.dissolve().iloc[0]['geometry']

In [32]:
# Determine if object is in NYC
df['in_nyc'] = df['geometry'].map(lambda x: nyc.contains(x))

In [33]:
# Limit ways to those in NYC
df = df.loc[df['in_nyc']]
df.drop(columns='in_nyc', inplace=True)

In [34]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 49480 entries, 17 to 1266771
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   name      49480 non-null  object  
 1   geometry  49480 non-null  geometry
dtypes: geometry(1), object(1)
memory usage: 1.1+ MB


In [35]:
boros

,boro_code,boro_name,shape_area,shape_leng,geometry
0,4,Queens,3040205594.95,900269.280485,"MULTIPOLYGON (((-73.82645 40.59053, -73.82642 ..."
1,3,Brooklyn,1934167333.21,728477.954082,"MULTIPOLYGON (((-73.86327 40.58388, -73.86381 ..."
2,5,Staten Island,1623631283.36,325924.002076,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ..."
3,1,Manhattan,636620785.519,359993.126318,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."
4,2,Bronx,1187193588.79,463868.599917,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."


In [36]:
qns = boros.iloc[0]['geometry']
bk = boros.iloc[1]['geometry']
si = boros.iloc[2]['geometry']
mtn = boros.iloc[3]['geometry']
bx = boros.iloc[4]['geometry']

In [37]:
# Determine if ways are in a particular boro
df['queens'] = df['geometry'].map(lambda x: qns.contains(x))
df['brooklyn'] = df['geometry'].map(lambda x: bk.contains(x))
df['staten_island'] = df['geometry'].map(lambda x: si.contains(x))
df['manhattan'] = df['geometry'].map(lambda x: mtn.contains(x))
df['bronx'] = df['geometry'].map(lambda x: bx.contains(x))

In [38]:
# Determine if a way can be contained in multiple boros (answer: no)
df[['queens', 'brooklyn', 'staten_island', 'manhattan', 'bronx']].sum(axis=1).value_counts()

1    49374
0      106
dtype: int64

In [39]:
def get_boro(row: pd.Series) -> str | None:
    if row['queens']:
        return 'queens'
    if row['brooklyn']:
        return 'brooklyn'
    if row['staten_island']:
        return 'staten island'
    if row['manhattan']:
        return 'manhattan'
    if row['bronx']:
        return 'bronx'
    return None

In [40]:
df['boro'] = df.apply(lambda row: get_boro(row), axis=1)
df.drop(columns=['queens', 'brooklyn', 'staten_island', 'manhattan', 'bronx'], inplace=True)

In [41]:
df.head(3)

,name,geometry,boro
17,Boerum Place,"LINESTRING (-73.98923 40.69098, -73.98919 40.6...",brooklyn
22546,West 106th Street,"LINESTRING (-73.96042 40.79821, -73.96071 40.7...",manhattan
22547,West 80th Street,"LINESTRING (-73.98200 40.78559, -73.98080 40.7...",manhattan


In [42]:
# Lowercase all names
df['name'] = df['name'].map(lambda x: x.lower())

In [43]:
df.head(3)

,name,geometry,boro
17,boerum place,"LINESTRING (-73.98923 40.69098, -73.98919 40.6...",brooklyn
22546,west 106th street,"LINESTRING (-73.96042 40.79821, -73.96071 40.7...",manhattan
22547,west 80th street,"LINESTRING (-73.98200 40.78559, -73.98080 40.7...",manhattan


In [45]:
# Merge streets within a boro with the same name
df = df.dissolve(['name', 'boro']).reset_index()

In [46]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9436 entries, 0 to 9435
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   name      9436 non-null   object  
 1   boro      9436 non-null   object  
 2   geometry  9436 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 221.3+ KB


In [47]:
# Write to GeoJSON
df.to_file('../data/nyc_osm.geojson', driver='GeoJSON')